In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U keras-tuner

In [21]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential, Model
from keras.layers import Conv1D, Dense, Flatten, Dropout, Input, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras import utils
import keras_tuner as kt

utils.set_random_seed(42)

In [22]:
# Define Parameters
LOOKBACK = 24
HORIZON = 24
N_SPLITS = 4
EPOCHS = 10

# Prepare data

In [ ]:
imp_folder = os.getenv("DATA_PATH", "./default_data_path/")
exp_folder = os.getenv("MODEL_PATH", "./default_model_path/")

df = pd.read_csv(imp_folder + 'cell_multivar.csv')

print(df.shape)
df.columns

(933661, 24)


Index(['timestamp', 'cell', 'bts', 'antenna', 'carrier', 'minRSSI',
       'pageSessions', 'ULvol', 'sessionDur', 'blocks', 'AnomalyDay',
       'anomaly', 'noise', 'Height', 'Azimuth', 'SectorsPerBts', 'NearbyBts',
       'Dist2Coast', 'ClusterId', 'CellsPerBts', 'OverallPageSessions',
       'OverallULvol', 'OverallSessionDur', 'OverallBlocks'],
      dtype='object')

In [24]:
temporal_X = ['pageSessions', 'ULvol', 'sessionDur']
static_X = ['Height', 'Azimuth', 'Dist2Coast', 'ClusterId',
            'CellsPerBts', 'OverallPageSessions', 'OverallULvol',
            'OverallSessionDur', 'OverallBlocks']

# Funcs

In [25]:
# Time series split function (Expanding Window)
def time_series_split(df, n_splits=N_SPLITS, test_size=0.2):
    df = df.sort_values('timestamp')
    test_split_index = int(len(df) * (1 - test_size))
    train_val_df = df.iloc[:test_split_index]
    test_df = df.iloc[test_split_index:]

    tscv = TimeSeriesSplit(n_splits=n_splits)
    splits = [(train_val_df.iloc[train_index], train_val_df.iloc[val_index]) for train_index, val_index in tscv.split(train_val_df)]
    return splits, test_df

In [26]:
# Sequence creation for multivariate time series
def create_sequences(df, lookback=LOOKBACK, horizon=HORIZON, static_X=static_X, temporal_X=temporal_X):
    X, y, anomaly, cell_id = [], [], [], []

    # Loop through each unique cell in the dataset
    for cell in df['cell'].unique():
        # Filter the dataframe for the current cell only
        cell_df = df[df['cell'] == cell]

        # Generate sequences within this cell's data
        for i in range(lookback, len(cell_df) - horizon + 1):
            # Lookback sequences with time-variant features
            X_seq = cell_df.iloc[i - lookback:i][['minRSSI'] + temporal_X].values

            # Repeat static features across lookback window and concatenate to time-variant features
            static_seq = cell_df.iloc[i][static_X].values  # Static features for this cell at a single timestep
            static_seq = np.tile(static_seq, (lookback, 1))  # Repeat to match lookback window length

            # Concatenate time-variant and time-invariant features
            X_combined = np.concatenate([X_seq, static_seq], axis=1)

            # Target horizon sequence
            y_seq = cell_df.iloc[i:i + horizon]['minRSSI'].values
            # Anomaly sequences for later evaluation
            anomaly_seq = cell_df.iloc[i:i + horizon]['anomaly'].values
            # Cell ID for each sequence
            cell_seq = cell_df.iloc[i:i + horizon]['cell'].values

            # Append sequences to output lists
            X.append(X_combined)
            y.append(y_seq)
            anomaly.append(anomaly_seq)
            cell_id.append(cell_seq)

    # Convert lists to numpy arrays for model input
    return np.array(X), np.array(y), np.array(anomaly), np.array(cell_id)

In [27]:
def scale_data_split(train_df, val_df, temporal_features=temporal_X, static_features=static_X):
    scaler_temporal = StandardScaler()
    scaler_static = MinMaxScaler()
    scaler_target = StandardScaler()

    # Scale time-variant features
    train_df[temporal_features] = scaler_temporal.fit_transform(train_df[temporal_features])
    val_df[temporal_features] = scaler_temporal.transform(val_df[temporal_features])

    # Scale time-invariant features
    train_df[static_features] = scaler_static.fit_transform(train_df[static_features])
    val_df[static_features] = scaler_static.transform(val_df[static_features])

    # Scale minRSSI separately (target variable)
    train_df['minRSSI'] = scaler_target.fit_transform(train_df[['minRSSI']])
    val_df['minRSSI'] = scaler_target.transform(val_df[['minRSSI']])

    return train_df, val_df, scaler_target, scaler_temporal, scaler_static

In [31]:
def tune_multi_cnn(X_train, y_train, X_val, y_val, max_epochs=EPOCHS):

    def build_tunable_cnn(hp):
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

        # Add CNN layers with filter size, kernel size, activation, and padding
        for i in range(hp.Int('num_conv_layers', 1, 4)):
            model.add(Conv1D(
                filters=hp.Choice(f'conv_filters_{i}', [16, 32, 64, 128]),
                kernel_size=hp.Choice(f'conv_kernel_size_{i}', [1, 3, 5, 7]),
                activation=hp.Choice(f'conv_activation_{i}', ['relu', 'tanh', 'linear']),
                padding=hp.Choice(f'conv_padding_{i}', ['valid', 'same']),
                kernel_regularizer=l2(hp.Choice('l2_regularizer', values=[1e-2, 1e-3, 1e-4]))
            ))

            # Optionally add MaxPooling layer
            if hp.Boolean(f'use_maxpooling_{i}'):
                model.add(MaxPooling1D(
                    pool_size=hp.Choice(f'maxpool_size_{i}', [2, 3]),
                    strides=hp.Choice(f'maxpool_stride_{i}', [1, 2])
                ))

        model.add(Flatten())

        # Add dense layers
        for j in range(hp.Int('num_dense_layers', 1, 3)):
            model.add(Dense(
                units=hp.Choice(f'dense_units_{j}', [32, 64, 128, 256]),
                activation=hp.Choice(f'dense_activation_{j}', ['relu', 'tanh', 'linear'])
            ))

            # Optionally add dropout for dense layers
            if hp.Boolean(f'use_dense_dropout_{j}'):
                dense_dropout_rate = hp.Float(f'dense_dropout_rate_{j}', 0.1, 0.3, step=0.1)
                model.add(Dropout(dense_dropout_rate))

        # Final output layer for predictions
        model.add(Dense(HORIZON))

        # Compile model
        model.compile(
            optimizer='adam',
            loss=hp.Choice('loss', ['mse', 'mae'])
        )

        return model

    # Instantiate the Hyperband tuner
    tuner = kt.Hyperband(
        hypermodel=build_tunable_cnn,
        objective='val_loss',
        max_epochs=max_epochs,
        factor=3,
        directory='/content/drive/MyDrive/Thesis/Thesis/cnn/multivar_tuning',
        project_name='1dcnn_multi_tuning'
    )

    # Perform the search
    tuner.search(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=max_epochs,
        verbose=1
    )

    # Retrieve best model and hyperparameters
    best_model = tuner.get_best_models(num_models=1)[0]
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0].values

    return best_model, best_params


# Run the tuning

In [29]:
splits, test_set = time_series_split(df, 4)

for i, (train, val) in enumerate(splits):
    print(f"Split {i + 1}:")
    print(f"  Train set shape: {train.shape}")
    print(f"  Validation set shape: {val.shape}")

print(f"Test set shape: {test_set.shape}")

Split 1:
  Train set shape: (149388, 24)
  Validation set shape: (149385, 24)
Split 2:
  Train set shape: (298773, 24)
  Validation set shape: (149385, 24)
Split 3:
  Train set shape: (448158, 24)
  Validation set shape: (149385, 24)
Split 4:
  Train set shape: (597543, 24)
  Validation set shape: (149385, 24)
Test set shape: (186733, 24)


In [30]:
scalers = {}

split1, split2, split3, split4 = splits
scaled_train, scaled_val, scaler_target, scaler_temporal, scaler_static = scale_data_split(split3[0], split3[1])
scalers = {
    'scaler_target': scaler_target,
    'scaler_temporal': scaler_temporal,
    'scaler_static': scaler_static
}

# Create sequences
X_train, y_train, _, _ = create_sequences(scaled_train, LOOKBACK, HORIZON)
X_val, y_val, _, _ = create_sequences(scaled_val, LOOKBACK, HORIZON)

# Convert to float32 to avoid data type issues
X_train, y_train = X_train.astype(np.float32), y_train.astype(np.float32)
X_val, y_val = X_val.astype(np.float32), y_val.astype(np.float32)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_train shape: (435139, 24, 13), y_train shape: (435139, 24)
X_val shape: (136366, 24, 13), y_val shape: (136366, 24)


In [32]:
# Tune and train the model
best_model, best_params = tune_multi_cnn(X_train, y_train, X_val, y_val)
print(best_params)

Trial 30 Complete [00h 05m 11s]
val_loss: 0.2650112807750702

Best val_loss So Far: 0.2604689598083496
Total elapsed time: 01h 03m 55s
{'num_conv_layers': 1, 'conv_filters_0': 32, 'conv_kernel_size_0': 3, 'conv_activation_0': 'relu', 'conv_padding_0': 'valid', 'l2_regularizer': 0.001, 'use_maxpooling_0': False, 'num_dense_layers': 3, 'dense_units_0': 32, 'dense_activation_0': 'linear', 'use_dense_dropout_0': False, 'loss': 'mae', 'maxpool_size_0': 2, 'maxpool_stride_0': 2, 'conv_filters_1': 128, 'conv_kernel_size_1': 1, 'conv_activation_1': 'tanh', 'conv_padding_1': 'same', 'use_maxpooling_1': False, 'conv_filters_2': 64, 'conv_kernel_size_2': 7, 'conv_activation_2': 'linear', 'conv_padding_2': 'valid', 'use_maxpooling_2': True, 'conv_filters_3': 64, 'conv_kernel_size_3': 7, 'conv_activation_3': 'linear', 'conv_padding_3': 'same', 'use_maxpooling_3': False, 'dense_dropout_rate_0': 0.2, 'dense_units_1': 64, 'dense_activation_1': 'relu', 'use_dense_dropout_1': True, 'maxpool_size_2': 3, 